In [7]:
import pandas as pd
import numpy as np

from helper.helper_functions import load_dataset, save_dataset, load_model, encode_nominal_features, encode_all_features, get_features_and_target

from scipy.stats import entropy
from scipy.special import kl_div
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Loading the datasets

In [8]:
data_original = load_dataset("../data/assignment2_income_cleaned.xlsx")
data_test = load_dataset("../data/assignment2_test.xlsx")

### Test Data Inspection

In [9]:
data_test.info()

In [10]:
# check the amount of missing values in the 'gave birth this year' column for the age bins (17-28, 28-38, 38-49, 49-65, 65-93) for females
female_data = data_test[data_test['sex'] == 'Female']
female_data['gave birth this year'][female_data['gave birth this year'].isnull()].groupby(
    pd.cut(female_data['age'], bins=[0, 28, 38, 49, 65, 93]), observed=False).size()

We impute the missing values in the 'ability to speak english' and 'gave birth this year' columns in the same manner as before.

In [11]:
data_test['ability to speak english'] = data_test['ability to speak english'].fillna(0)
data_test['gave birth this year'] = data_test['gave birth this year'].fillna('No')

### Test Data Distribution Discrepancy

Using the KL divergence to measure the distribution discrepancy between the training and test sets, we want to identify the features that have the highest discrepancy and calculate the mean discrepancy between the two datasets.

In [13]:
def compute_kl_divergence(train_set: pd.DataFrame, test_set: pd.DataFrame, numerical_features: list[str]):
    """
    Compute KL divergence between the distributions of features in the training and test sets.
    :param train_set: DataFrame containing features of the training set.
    :param test_set: DataFrame containing features of the test set.
    :param numerical_features: List of numerical features.
    :return: Array of KL divergence values for each feature in Series format.
    """

    kl_divergences = []
    train_set = train_set[test_set.columns]
    for feature in test_set.columns:  # Use test set columns since it has no target label
        
        # code below seems to produce slightly different results.
        # if feature not in numerical_features:  # Handle categorical features
        #     train_dist = train_set[feature].value_counts(normalize=True)
        #     test_dist = test_set[feature].value_counts(normalize=True)
        # 
        #     # Ensure all possible values are represented in both distributions
        #     all_values = set(train_dist.index) | set(test_dist.index)
        #     for value in all_values:
        #         if value not in train_dist.index:
        #             train_dist[value] = 0.000001  # Add a small non-zero count for missing value
        #         if value not in test_dist.index:
        #             test_dist[value] = 0.000001  # Add a small non-zero count for missing value
        # else:  # Handle numerical features
        #     train_dist, _ = np.histogram(train_set[feature], bins=10, density=True)
        #     test_dist, _ = np.histogram(test_set[feature], bins=10, density=True)
        # 
        # assert train_dist.shape == test_dist.shape
        # 
        # # Normalizing
        # train_dist /= np.sum(train_dist)
        # test_dist /= np.sum(test_dist)
        # 
        # # KL divergence
        # kl_divergence = entropy(train_dist, test_dist)
        # kl_divergences.append(kl_divergence)
        
        kl_divergences.append(np.sum(kl_div(train_set[feature].value_counts(normalize=True).sort_index(), test_set[feature].value_counts(normalize=True).sort_index())))

    return pd.Series(kl_divergences, index=test_set.columns)


def aggregate_kl_divergence(kl_divergences: np.ndarray | pd.Series):
    """
    Aggregate KL divergence values across all features.
    :param kl_divergences: Array of KL divergence values for each feature.
    :return: Overall measure of distribution discrepancy (in this case the mean value).
    """
    if isinstance(kl_divergences, np.ndarray):
        return np.mean(kl_divergences)
    elif isinstance(kl_divergences, pd.Series):
        return kl_divergences.mean()

In [14]:
data_original_train, data_original_val = train_test_split(data_original, test_size=0.2, random_state=42)
kl_divergences = compute_kl_divergence(data_original, data_test, ['age', 'workinghours'])

In [15]:
kl_divergences_df = pd.DataFrame({'KL Divergence': kl_divergences})
kl_divergences_df.index = kl_divergences.index  # Set DataFrame index to match Series index
kl_divergences_df = kl_divergences_df.sort_values(by='KL Divergence', ascending=False)
kl_divergences_df

In [16]:
aggregate_kl_divergence(kl_divergences)

### Test Data Feature Encoding

Here, we encode the features of the test data using the same encoding as the training data. We exclude the columns that were not used in the model.

In [17]:
# drop columns that our model does not use
columns_to_exclude = ['sex', 'gave birth this year', 'marital status']
data_test_sexexcl = data_test.drop(columns=columns_to_exclude)

In [18]:
nominal_features_lc = list(
    {'workclass', 'marital status', 'gave birth this year', 'sex', 'occupation'} - set(
        columns_to_exclude))  # low cardinality features
nominal_features_hc = []

# Encoded test set
data_test_encoded = encode_nominal_features(data_test_sexexcl, nominal_features_lc, nominal_features_hc)

### Generate predictions for test data using the best model

We load the best model and generate predictions for the test data. We also save the predictions to a .xlsx file.

In [19]:
# load the best model
model = load_model('../output/saved_models/dt_model_sexexcl_ffs_tuned_fair.joblib')

In [20]:
# Get the feature names used during training
train_features = model.feature_names_in_
data_test_reordered = data_test_encoded[train_features]
data_test_encoded = data_test_reordered
# predict the test data
y_pred = model.predict(data_test_encoded)
# save the predictions
y_pred = pd.DataFrame(y_pred, columns=['income'])
save_dataset(y_pred, '../output/test_predictions/best_model_predictions.xlsx', index=True)

In [21]:
p = load_dataset('../output/test_predictions/best_model_predictions.xlsx')
print(len(p), len(y_pred)) # the lengths should be the same
# see if the predictions were saved correctly
differing_values = (y_pred['income'] != p['income']).sum()

Little check to see if the predictions were saved correctly.

In [22]:
differing_values

### Inspection of the predictions

We inspect the predictions to see the distribution of low and high income predictions. We do the same per sex.

In [23]:
# Count amount of low (0) and high (1) income predictions
predicted_1 = (y_pred == 1).sum()
predicted_0 = (y_pred == 0).sum()

# Create a DataFrame
predictions_df = pd.DataFrame({
    'Prediction': [1, 0],
    'Count': [predicted_1, predicted_0]
})

In [24]:
y_pred.value_counts()

In [25]:
# Filter predictions for males
male_predictions = y_pred[data_test['sex'] == 'Male']

male_predicted_1 = (male_predictions == 1).sum()
male_predicted_0 = (male_predictions == 0).sum()

# Filter predictions for females
female_predictions = y_pred[data_test['sex'] == 'Female']

female_predicted_1 = (female_predictions == 1).sum()
female_predicted_0 = (female_predictions == 0).sum()

predictions_df_mf = pd.DataFrame({
    'Sex': ['Male', 'Female'],
    'Predicted_1': [male_predicted_1, female_predicted_1],
    'Predicted_0': [male_predicted_0, female_predicted_0]
})

In [26]:
predictions_df_mf

In [27]:
columns_to_exclude = ['sex', 'gave birth this year', 'marital status']
X_original, y_original = get_features_and_target(data_original, 'income')
X_original = X_original.drop(columns=columns_to_exclude)
X_original_encoded, y_original_encoded = encode_all_features(X_original, y_original, columns_to_exclude)
X_train, X_test, y_train, y_test = train_test_split(X_original_encoded, y_original_encoded, test_size=0.2, random_state=42)

### Estimating the accuracy

We will express the accuracy of our best model (on the validation set) as a weighted sum of accuracies of the feature values. The next step is then to calculate the feature value distribution for the test set, so we can get those probabilities. We can plug in these probabilities (corresponding to their value of course) in the formula using the same accuracies as before, since we assume the accuracies stay roughly the same for our model and only the value distribution changes. Ultimately, we get the weighted accuracy for our test set.

In [28]:
def compute_accuracy_by_feature_value(model: any, feature_name: str, X_test: pd.DataFrame, y_test: pd.Series):
    """
    Compute the accuracy of the model on the test set for each unique value of a feature.
    :param model: model
    :param feature_name: name of the feature for the weighted accuracy
    :param X_test: validation set features
    :param y_test: validation set target
    :return: dictionary of (prob, accuracy) for each unique value of the feature
    """
    feature_values = X_test[feature_name].unique()
    accuracies = {}
    for value in feature_values:
        # Filter the data for the current feature value
        subset_data = X_test[X_test[feature_name] == value]
        # Get the probability of the current feature value
        prob = len(subset_data) / len(X_test)
        y_subset = y_test.loc[subset_data.index]
        
        # Get the feature names used during training
        train_features = model.feature_names_in_
        data_test_reordered = subset_data[train_features]
        subset_data = data_test_reordered
    
        y_pred = model.predict(subset_data)
        accuracy = accuracy_score(y_subset, y_pred)
        
        accuracies[value] = prob, accuracy
        
    assert 0.99 < sum([prob for prob, _ in accuracies.values()]) <= 1
    return accuracies

In [29]:
feature_name = 'workinghours'
feature_values = X_test[feature_name].unique()

In [30]:
accuracy_by_value = compute_accuracy_by_feature_value(model, feature_name, X_test, y_test)
# print(accuracy_by_value) # uncomment to see the probabilities (~weights) and accuracies

Below, we calculate the weighted accuracy for the 'workinghours' feature for the validation set (from the original dataset).

In [31]:
sum_of_prob = sum([accuracy_by_value[key][0] for key in accuracy_by_value])
# get the weighted accuracy by summing the product of the probability and accuracy
weighted_accuracy = sum([accuracy_by_value[key][0] * accuracy_by_value[key][1] for key in accuracy_by_value]) / sum_of_prob
print("Weighted Accuracy:", weighted_accuracy)

We can then calculate the weighted accuracy for the 'workinghours' feature for the test set.

In [32]:
# Next, calculate the probability distribution of feature values in X_test
feature_distribution = data_test[feature_name].value_counts(normalize=True)
# print(feature_distribution) # uncomment to see the probabilities

weighted_accuracy = 0
for value in feature_values:
    # Get the probability and accuracy of the current feature value in test data
    prob = feature_distribution.get(value, 0.0000001)
    accuracy = accuracy_by_value.get(value, (prob, weighted_accuracy))[1]  # 0 as default accuracy if value not found
    # Update the weighted accuracy by adding the product of probability and accuracy
    weighted_accuracy += prob * accuracy

# Finally, print or return the weighted accuracy
print("Weighted Accuracy:", weighted_accuracy)